In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/constitutionofindiasquadformat/constitution_train.json
/kaggle/input/constitutionofindiasquadformat/constitution_eval.json
/kaggle/input/document/documentCOI.txt
/kaggle/input/weights/finetuned_epoch10.h5
/kaggle/input/weights/finetune.h5
/kaggle/input/weights/finetuned_epoch1.h5


In [5]:
!pip install tokenizers
!pip install transformers

In [6]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
from tqdm import tqdm as tqdm


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
def get_json_file(path):
    f = open(path)
    return json.load(f)

In [8]:
#loading json files into the notebook
raw_train_data = get_json_file('/kaggle/input/constitutionofindiasquadformat/constitution_eval.json')
raw_eval_data = get_json_file('/kaggle/input/constitutionofindiasquadformat/constitution_train.json')

In [9]:
raw_eval_data

{'data': [{'paragraphs': [{'qas': [{'question': 'how many ways a new state can be formed?',
       'id': 916574,
       'answers': [{'answer_id': 799765,
         'document_id': 1442838,
         'question_id': 916574,
         'text': 'a) form a new State by separation of territory from any State or by uniting two or more States or parts of States or by uniting any territory to a part of any State; ',
         'answer_start': 27,
         'answer_end': 193,
         'answer_category': None}],
       'is_impossible': False},
      {'question': 'who recommends the bill for this purpose?',
       'id': 916577,
       'answers': [{'answer_id': 799784,
         'document_id': 1442838,
         'question_id': 916577,
         'text': 'Bill for the purpose shall be introduced in either House of Parliament except on the recommendation of the President ',
         'answer_start': 356,
         'answer_end': 473,
         'answer_category': None}],
       'is_impossible': False}],
     'context

In [10]:
corpus = []

for i in range( len(raw_eval_data["data"]) ):
    for j in range( len(raw_eval_data["data"][i]["paragraphs"])):

        corpus.append(raw_eval_data["data"][i]["paragraphs"][j]["context"])
        

In [11]:
for i in range( len(raw_train_data["data"]) ):
    for j in range( len(raw_train_data["data"][i]["paragraphs"])):

        corpus.append(raw_train_data["data"][i]["paragraphs"][j]["context"])

In [12]:
corpus

['3. Parliament may by law— (a) form a new State by separation of territory from any State or by uniting two or more States or parts of States or by uniting any territory to a part of any State; (b) increase the area of any State; (c) diminish the area of any State; (d) alter the boundaries of any State; (e) alter the name of any State: 4[Provided that no Bill for the purpose shall be introduced in either House of Parliament except on the recommendation of the President and unless, where the proposal contained in the Bill affects the area, boundaries or name of any of the States 1***, the Bill has been referred by the President to the Legislature of that State for expressing its views thereon within such period as may be specified in the reference or within such further period as the President may allow and the period so specified or allowed has expired.] 2 [Explanation I.—In this article, in clauses (a) to (e), "State\'\' includes a Union territory, but in the proviso, "State\'\' does

In [13]:
import spacy
import re

nlp = spacy.load('en_core_web_sm')

def lowercase(string):
    return string.lower()

def remove_non_alpha(string):
    return re.sub('[^a-z]', ' ', string)

def remove_extra_spaces(string):
    return re.sub(' +', ' ', string)

def tokenize(string):
    doc = nlp(string)
    return [token for token in doc]

def lemmatize(tokens):
    return [token.lemma_.lower() for token in tokens]

def preprocess(string):
    string = lowercase(string)
    string = remove_non_alpha(string)
    string = remove_extra_spaces(string)
    tokens = tokenize(string)
    tokens = lemmatize(tokens)
    return tokens








In [14]:
print()

In [15]:
processed = []

for i in corpus:
    processed.append(preprocess(i))

In [16]:
print(corpus[4])

27. No person shall be compelled to pay any taxes, the proceeds of which are specifically appropriated in payment of expenses for the promotion or maintenance of any particular religion or religious denomination. 28. (1) No religious instruction shall be provided in any educational institution wholly maintained out of State funds. (2) Nothing in clause (1) shall apply to an educational institution which is administered by the State but has been established under any endowment or trust which requires that religious instruction shall be imparted in such institution. (3) No person attending any educational institution recognised by the State or receiving aid out of State funds shall be required to take part in any religious instruction that may be imparted in such institution or to attend any religious worship that may be conducted in such institution or in any premises attached thereto unless such person or, if such person is a minor, his guardian has given his consent thereto.



In [17]:
!pip install rank_bm25

from rank_bm25 import BM25Okapi

bm25 = BM25Okapi(processed)

In [18]:
query = "who can register the person residing outside India as citizen of india"
query = preprocess(query)
bm25.get_top_n(query, corpus, n=1)

['8. Notwithstanding anything in article 5, any person who or either of whose parents or any of whose grandparents was born in India as defined in the Government of India Act, 1935 (as originally enacted), and who is ordinarily residing in any country outside India as so defined shall be deemed to be a citizen of India if he has been registered as a citizen of India by the diplomatic or consular representative of India in the country where he is for the time being residing on an application made by him therefor to such diplomatic or consular representative, whether before or after the commencement of this Constitution, in the form and manner prescribed by the Government of the Dominion of India or the Government of India.\n']

In [31]:


def read_file(filename):
    with open(filename, 'r') as f:
        return f.read()

def split_documents(string, separator):
    return string.split(separator)

def remove_newlines(document):
    return re.sub('\n', ' ', document).strip()

def preprocess_corpus(corp, separator):
    preprocessed_corpus = corpus
    for document in corp:
        document = remove_newlines(document)
        if document.startswith(' '):
            document = document[1:]
        preprocessed_corpus.append(document)
    return preprocessed_corpus

def create_corpus_from_file(filename, separator='//'):
    string = read_file(filename)
    corp = split_documents(string, separator)
    preprocessed_corpus = preprocess_corpus(corp, separator)
    return preprocessed_corpus

In [32]:
new_corpus=create_corpus_from_file('/kaggle/input/document/documentCOI.txt')

In [33]:
print(new_corpus)

['3. Parliament may by law— (a) form a new State by separation of territory from any State or by uniting two or more States or parts of States or by uniting any territory to a part of any State; (b) increase the area of any State; (c) diminish the area of any State; (d) alter the boundaries of any State; (e) alter the name of any State: 4[Provided that no Bill for the purpose shall be introduced in either House of Parliament except on the recommendation of the President and unless, where the proposal contained in the Bill affects the area, boundaries or name of any of the States 1***, the Bill has been referred by the President to the Legislature of that State for expressing its views thereon within such period as may be specified in the reference or within such further period as the President may allow and the period so specified or allowed has expired.] 2 [Explanation I.—In this article, in clauses (a) to (e), "State\'\' includes a Union territory, but in the proviso, "State\'\' does

In [34]:
processed = []

for i in corpus:
    processed.append(preprocess(i))

In [35]:
bm25 = BM25Okapi(processed)

In [36]:
query = "Lunguanges in which member can express themselves in parliament"
query = preprocess(query)
bm25.get_top_n(query, corpus, n=1)

['120. (1) Notwithstanding anything in  Part XVII, but subject to the provisions of article 348, business in Parliament shall be transacted in Hindi or in English: Provided that the Chairman of the Council of States or Speaker of the House of the People, or person acting as such, as the case may be, may permit any member who cannot adequately express himself in Hindi or in English to address the House in his mother-tongue. (2) Unless Parliament by law otherwise provides, this article shall, after the expiration of a period of fifteen years from the commencement of this Constitution, have effect as if the words "or in English" were omitted therefrom.']